# Tingkat Inflasi Harga Konsumen Nasional Tahunan (Y-on-Y)

[Source](https://www.bps.go.id/id/statistics-table/1/OTE1IzE=/-tingkat-inflasi-harga-konsumen-nasional-tahunan--y-on-y---sup-1--sup---2022-100-.html)

In [ ]:
import html
import json
from calendar import monthrange
from io import StringIO
from os import getenv

import httpx
import lxml.html
import numpy as np
import pandas as pd

In [ ]:
resp: httpx.Response = httpx.get(
    "https://webapi.bps.go.id/v1/api/view/domain/{DOMAIN}/model/{MODEL}/lang/{LANG}/id/{ID}/key/{KEY}/".format(
        KEY=getenv("BPS_KEY"),
        LANG="ind",
        DOMAIN="0000",  # Pusat
        MODEL="statictable",  # Static Table
        ID=915,  # Tingkat Inflasi Harga Konsumen Nasional Tahunan (Y-on-Y) 1 (2022=100)
    )
)

with open("datalake.inflation.json", "wb") as json_file:
    json_file.write(resp.content)

data: dict = resp.json()["data"]

In [ ]:
with open("datalake.inflation.json") as json_file:
    data: dict = json.load(json_file)["data"]

In [ ]:
title: str = lxml.html.fromstring(data["title"]).text.strip()
df: pd.DataFrame = pd.read_html(
    io=StringIO(
        html.unescape(data["table"])
    ),
    header=2,
    index_col=0,
    decimal=",",
    thousands=".",
    flavor="lxml",
)[0][:12]
df.style.set_caption(title).format(precision=2)

In [ ]:
df_flat_table: pd.DataFrame = df.reset_index().rename(columns={"index": "Bulan"}).melt(
    id_vars="Bulan",
    var_name="Tahun",
    value_name="inflation",
)
df_flat_table["date"] = (
    pd
    .concat(
        [
            df_flat_table["Tahun"].astype(int),
            df_flat_table["Bulan"].map(
                {
                    "Januari": 1,
                    "Februari": 2,
                    "Maret": 3,
                    "April": 4,
                    "Mei": 5,
                    "Juni": 6,
                    "Juli": 7,
                    "Agustus": 8,
                    "September": 9,
                    "Oktober": 10,
                    "November": 11,
                    "Desember": 12,
                }
            )
        ],
        axis=1,
    )
    .apply(
        lambda row: pd.to_datetime(
            "{year}-{month}-{day}".format(
                year=row["Tahun"],
                month=row["Bulan"],
                day=monthrange(row["Tahun"], row["Bulan"])[1],
            )
        ),
        axis=1,
    )
    .dt
    .date
)
df_flat_table["inflation"] = df_flat_table["inflation"].astype(float)

del df_flat_table["Bulan"]
del df_flat_table["Tahun"]

df_flat_table.set_index("date", inplace=True)

df_flat_table

In [ ]:
df_flat_table.plot(
    xlabel="Waktu",
    ylabel="Inflasi (%)",
    legend=False,
    title=title,
)

In [ ]:
with open("datamart.inflation.json", "w") as json_file:
    json.dump(
        dict(
            title=title,
            data=df_flat_table.replace({np.nan: None}).reset_index().to_dict("records"),
        ),
        json_file,
        default=str,
    )